# Load Model for Inference

In [ ]:
import os
import sys
import pickle, types
import types
import importlib.util

import pandas as pd


from pathlib import Path
from IPython.display import Image

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from data_processing import DataProcessing
from feature_extraction import TfidfFeatureExtraction, SpacyFeatureExtraction

# Absolute path to your local classification_models.py file
project_root = "/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions"
module_path = os.path.join(project_root, "classification_models.py")

# Dynamically load the module so it overrides any installed package
spec = importlib.util.spec_from_file_location("classification_models", module_path)
classification_models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(classification_models)

# Inject into sys.modules so pickle uses this
sys.modules["classification_models"] = classification_models

# ✅ Now you can use the classes
SkLearnSGDClassifier = classification_models.SkLearnSGDClassifier
SkLearnPerceptronModel = classification_models.SkLearnPerceptronModel
EvaluationMetric = classification_models.EvaluationMetric

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# custom_sentences = [
#     "Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027.",
#     "Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026.",
#     "There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029.",
#     "On Thursday, April 10, 2025, Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026.",
#     "Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027.",
#     "It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028.",
#     "Hey, how are you?",
#     "Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028, which Raj stated on Monday, December 16, 2024.",
#     "Malique Mell, on Monday, December 16, 2024, predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90%, reaching $10 million in FY 2028.",
#     "Raj Jensen predicts that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028.",
#     "The weather today is sunny with a chance of rain in the evening.",
#     "I enjoy reading books and watching movies during my free time."
# ]

"Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027, Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026., There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029., On Thursday April 10 2025 Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026., Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027., It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028., Hey how are you?, Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028 which Raj stated on Monday, December 16 2024., Malique Mell on Monday December 16 2024 predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90% reaching $10 million in FY 2028., Raj Jensen predicts that the earnings before interest and taxes EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028., The weather today is sunny with a chance of rain in the evening. I enjoy reading books and watching movies during my free time."


custom_sentences = "Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027, Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026., There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029., On Thursday April 10 2025 Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026., Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027., It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028., Hey how are you?, Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028 which Raj stated on Monday, December 16 2024., Malique Mell on Monday December 16 2024 predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90% reaching $10 million in FY 2028., Raj Jensen predicts that the earnings before interest and taxes EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028., The weather today is sunny with a chance of rain in the evening. I enjoy reading books and watching movies during my free time."
custom_sentences

'Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027, Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026., There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029., On Thursday April 10 2025 Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026., Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027., It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028., Hey how are you?, Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028 which Raj stated on Monday, December 16 2024., Malique Mell on Monday December 16 2024 predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90% reaching $10 million in FY 2028., Raj Jensen pr

In [5]:
input_list = custom_sentences.split(",")
input_list

['Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027',
 ' Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026.',
 ' There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029.',
 ' On Thursday April 10 2025 Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026.',
 ' Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027.',
 ' It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028.',
 ' Hey how are you?',
 ' Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028 which Raj stated on Monday',
 ' December 16 2024.',
 ' Malique Mell on Monday December 16 2024 predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90% reaching $

In [6]:
user_input_df = pd.DataFrame(input_list, columns=['Base Sentence'])
user_input_df

,Base Sentence
0,Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027
1,Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026.
2,There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029.
3,On Thursday April 10 2025 Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026.
4,Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027.
5,It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028.
6,Hey how are you?
7,Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028 which Raj stated on Monday
8,December 16 2024.
9,Malique Mell on Monday December 16 2024 predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90% reaching $10 million in FY 2028.


In [7]:
max_features = None

tf_idf_feature_extractor = TfidfFeatureExtraction(user_input_df, 'Base Sentence')
custom_sentences_features = tf_idf_feature_extractor.word_feature_extraction(max_features=512)
custom_sentences_features

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 220 stored elements and shape (12, 117)>

In [8]:
MODEL_PATH = os.path.join(notebook_dir, '../models/', 'sgd_model_117.pkl')
with open(MODEL_PATH, 'rb') as f:
    SkLearnSGDClassifier = pickle.load(f)

In [9]:
custom_sentences_predictions = SkLearnSGDClassifier.predict(custom_sentences_features)
custom_sentences_predictions

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


0     1
1     1
2     0
3     0
4     0
5     0
6     1
7     1
8     0
9     1
10    1
11    1
dtype: int64

In [10]:
DataProcessing.join_predictions_with_sentences(user_input_df, custom_sentences_predictions, SkLearnSGDClassifier)


,Base Sentence,SDG Classifier Prediction
0,Emily Chen forecasts that the net profit at Tesla (TSLA) will decrease by 15% to $5 billion in FY 2027,1
1,Raj Patel speculates that the stock price of ExxonMobil (XOM) could rise by 8% to $120 by Q4 of 2026.,1
2,There is a high probability that the revenue at Microsoft (MSFT) will reach $200 billion in FY 2029.,0
3,On Thursday April 10 2025 Sarah Lee envisions that the operating income at Apple (AAPL) will increase by 10% to $80 billion in FY 2026.,0
4,Michael Brown predicts that the dividend yield at Chevron (CVX) will rise to 5% by Q3 of 2027.,0
5,It is anticipated that the market share of Alphabet (GOOGL) will grow by 3% in FY 2028.,0
6,Hey how are you?,1
7,Joe Hall thinks that the earnings before interest and taxes (EBIT) at 3M (MMM) will decrease by 90% to $10 million in FY 2028 which Raj stated on Monday,1
8,December 16 2024.,0
9,Malique Mell on Monday December 16 2024 predicted that the earnings before interest and taxes (EBIT) at 3M (MMM) will drop by 90% reaching $10 million in FY 2028.,1
